# About this notebook (LB Score : 0.546 )
 ### pytorch BERTの初心者部屋です。English version is here. https://www.kaggle.com/chumajin/pytorch-bert-beginner-s-room
 
 
 ### スコアを目指すよりも、以下のことを目的としています。
 
*   理解を深める。
*   言語処理でも数字のニューラルネットワークと同じように処理する。
        
       pytorchのニューラルネットワーク組んだことない方はまずはこちらをお勧めします。
       
       https://www.kaggle.com/chumajin/pytorch-neural-network-starter-detail
       
       

      
        


*   BERTの雛形を作成する。
  

  ### 少しでもお役に立てれば幸いです。upvoteよろしくお願いいたします !
  ### あと、いつもupvoteしていただける方ありがとうございます。
  
  
  -----ご参考--------------------------------------------------
  ### inference onlyのnotebookはこちらです。ver 7の結果（これより少し良いスコア)をuploadしております。
  ### (ver8まではこのnotebookではtrainも入るため、submitに時間がかかりすぎてsubmit errorが起こっていました)

inference onlyのnotebookはこちら　　https://www.kaggle.com/chumajin/inference-for-pytorch-bert-beginner-s-room?scriptVersionId=62477318

inference onlyにするのは、submitが早いのと、internet onにしてしまった場合にsubmitできます。

--------とは言っても、ある程度スコア出ないと公開できなかったので、以下工夫した点のメモです --------

  (初心者用と言っているくせにマニアック。後ほど解説します。)
* BertModelを使用するよりも、BertForSequenceClassificationを使用した方が私はスコアが出たので、それを採用しました。
* Bertのfine-tuningの不安定性について、 https://ai-scholar.tech/articles/bert/bert-fine-tuning を参考にパラメータをある程度採用しました。
     
    ※　Large-modelで20 epoch,5 k-foldsは時間かかるので、そこだけ、base uncased model採用しました。
   
---------使えそうなやつ-------------

https://zenn.dev/koukyo1994/articles/9b1da2482d8ba1

# Overview of model
- RoBERta
- batchsize:, sen_length:, gradient_accumulation_steps = 
- BertForSeqenceClassification
- no data augumentation

# transfer data to workspace

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt update
!sudo apt-get install p7zip-full p7zip-rar

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [3]:
!7z x /content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/input/commonlitreadabilityprize.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/kaggle/C . _Readability_Priize/data/input/                                                                           1 file, 1186779 bytes (1159 KiB)

Extracting archive: /content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/input/commonlitreadabilityprize.zip
--
Path = /content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/input/commonlitreadabilityprize.zip
Type = zip
Physical Size = 1186779

  0%    
Would you like to replace the existing file:
  Path:     ./sample_submission.csv
  Size:     108 bytes (1 KiB)
  Modified: 2021-05-02 22:19:12
with the 

In [4]:
!7z x /content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/input/archive.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/kaggle/C . _Readability_Priize/data/input/                                                                           1 file, 3346761599 bytes (3192 MiB)

Extracting archive: /content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/input/archive.zip
--
Path = /content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/input/archive.zip
Type = zip
Physical Size = 3346761599

  0%    
Would you like to replace the existing file:
  Path:     ./roberta-base/config.json
  Size:     481 bytes (1 KiB)
  Modified: 2021-05-06 12:26:42
with the file from archive:
  Path: 

In [5]:
!pip install -U transformers

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.6.1)


# Data Loading

In [6]:
import numpy as np
import pandas as pd 
import os

In [7]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [8]:
display(train.head())
display(test.head())

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


# Quick EDA

In [9]:
# excerptの0番目の文章例
train["excerpt"][0]

'When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes.'

In [10]:
categorical_cols = ["license"]
numeric_cols = ["standard_error"]


# Directory setting

In [11]:
filename = "001"
MODEL_DIR = "/content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/models/{}".format(filename)
LOG_DIR = "/content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/logs/{}".format(filename)
OUTPUT_DIR = "/content/drive/MyDrive/kaggle/CommonLit_Readability_Priize/data/output/{}".format(filename)

for d in [MODEL_DIR, LOG_DIR, OUTPUT_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

# CFG

In [12]:
# ====================================================
# CFG
# ====================================================
class CFG:
    apex=False
    debug=False
    print_freq=50
    num_workers=4
    model_name='roberta-base'
    sen_length=314
    scheduler='LinearSchedule' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts', 'LinearSchedule']
    epochs=20
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    #T_max=6 # CosineAnnealingLR
    #T_0=6 # CosineAnnealingWarmRestarts
    lr=2e-5
    min_lr=1e-6
    batch_size=16
    weight_decay=1e-2
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=27
    target_size=1 # 回帰でやるから1で
    target_col='target'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    
if CFG.debug:
    CFG.epochs = 1
    CFG.sen_length = 10
    train = train.sample(n=100, random_state=CFG.seed)

# Library

In [13]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('/content/roberta-base')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
from tqdm import tqdm

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from torch.cuda.amp import autocast, GradScaler# GPUでの高速化。

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

import warnings
warnings.simplefilter('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Utils

In [14]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = np.sqrt(mean_squared_error(y_true, y_pred))
    return score


def init_logger(log_file=LOG_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=27):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

# CV split

In [15]:
train = train.sort_values("target").reset_index(drop=True)
train["fold"] = train.index % 5

display(train.groupby(['fold']).size())

fold
0    567
1    567
2    567
3    567
4    566
dtype: int64

# Dataset

In [16]:
tokenizer = transformers.RobertaTokenizer.from_pretrained("/content/roberta-base")

In [17]:
class RobertaDataSet(Dataset):
    
    def __init__(self,sentences,targets):
        
        self.sentences = sentences
        self.targets = targets
        
    def __len__(self):
        
        return len(self.sentences)
    
    def __getitem__(self,idx):
        
        sentence = self.sentences[idx]
        sentence = sentence.replace('\n', '')
        
        roberta_sens = tokenizer.encode_plus(
                            sentence,
                            add_special_tokens = True, 
                            max_length = CFG.sen_length, # 上で314に設定しています
                            truncation=True,
                            pad_to_max_length = True, 
                            return_attention_mask = True
                            )

        ids = torch.tensor(roberta_sens['input_ids'], dtype=torch.long)
        mask = torch.tensor(roberta_sens['attention_mask'], dtype=torch.long)
                 
        target = torch.tensor(self.targets[idx],dtype=torch.float)
        
        return {
                'ids': ids,
                'mask': mask,
                'targets': target
            }

# MODEL

In [18]:
# # ====================================================
# # MODEL
# # ====================================================
# class CustomModel(nn.Module):
#     def __init__(self, cfg, pretrained=False):
#         super().__init__()
#         self.cfg = cfg
#         self.model = transformers.BertForSequenceClassification.from_pretrained("../input/{}".format(cfg.model_name),num_labels=1)

#     def forward(self, x):
#         output = self.model(x)
#         return output

# LOSS

In [19]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

# Helper functions

In [20]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [21]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    scaler = torch.cuda.amp.GradScaler() # GPUでの高速化。
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    allpreds = []
    alltargets = []

    model.train()
    torch.backends.cudnn.benchmark = True
    start = end = time.time()

    allpreds = []
    alltargets = []

    for step, a in enumerate(train_loader):

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():

            ids = a["ids"].to(device,non_blocking=True) # non_blocking=TrueでPinned MemoryからGPUに転送中もCPUが動作できるらしい。
            mask = a["mask"].to(device,non_blocking=True)
            batch_size = ids.size(0)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)

            target = a["targets"].to(device,non_blocking=True)

            loss = criterion(output,target)


            # スコア習得用
            losses.update(loss.item(), batch_size)
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())

        scaler.scale(loss).backward() # ロスのバックワード
        scaler.step(optimizer) # オプティマイザーの更新
        scaler.update() # スケーラーの更新
        
        del loss # ここでlossを消した方がGPUのメモリの無駄な部分を消せるらしい。https://tma15.github.io/blog/2020/08/22/pytorch%E4%B8%8D%E8%A6%81%E3%81%AB%E3%81%AA%E3%81%A3%E3%81%9F%E8%A8%88%E7%AE%97%E3%82%B0%E3%83%A9%E3%83%95%E3%82%92%E5%89%8A%E9%99%A4%E3%81%97%E3%81%A6%E3%83%A1%E3%83%A2%E3%83%AA%E3%82%92%E7%AF%80%E7%B4%84/

        scheduler.step() # 学習率の更新
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
#                   'Grad: {grad_norm:.4f}  '
#                   'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
#                    grad_norm=grad_norm,
#                    lr=scheduler.get_lr()[0],
                   ))



    # データローダーの合併

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)

    # ロス関数はモニター用だけど、一応保管

    # Score with rmse
    train_rme_loss = np.sqrt(mean_squared_error(alltargets,allpreds))

    return losses.avg,train_rme_loss

In [22]:
# def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
#     scaler = torch.cuda.amp.GradScaler() # GPUでの高速化。
#     batch_time = AverageMeter()
#     data_time = AverageMeter()
#     losses = AverageMeter()
#     scores = AverageMeter()
#     allpreds = []
#     alltargets = []
    
#     # switch to train mode
#     model.train()
#     start = end = time.time()
# #     torch.backends.cudnn.benchmark = True
#     global_step = 0

#     for step, a in enumerate(train_loader):
#         optimizer.zero_grad()
        
#         with autocast():
#             ids = a["ids"].to(device,non_blocking=True) # non_blocking=TrueでPinned MemoryからGPUに転送中もCPUが動作できるらしい。
#             mask = a["mask"].to(device,non_blocking=True)
#             batch_size = ids.size(0)
        
#             y_preds = model(ids,mask)["logits"]
#             target = a["targets"].to(device,non_blocking=True)
#             loss = criterion(y_preds.squeeze(-1), target)
        
#             # record loss
#             losses.update(loss.item(), batch_size)
#             allpreds.append(y_preds.detach().cpu().numpy())
#             alltargets.append(target.detach().squeeze(-1).cpu().numpy())

# #         if CFG.gradient_accumulation_steps > 1:
# #             loss = loss / CFG.gradient_accumulation_steps
# #         if CFG.apex:
# #             scaler.scale(loss).backward()
# #         else:
# #             loss.backward()

#         grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
# #         if (step + 1) % CFG.gradient_accumulation_steps == 0:
# #             if CFG.apex:
# #                 scaler.step(optimizer)
# #                 scaler.update()
# #             else:
# #                 optimizer.step()
# #             optimizer.zero_grad()
# #             global_step += 1
        
#         scaler.scale(loss).backward() # ロスのバックワード
#         scaler.step(optimizer) # オプティマイザーの更新
#         scaler.update() # スケーラーの更新
                    
#         del loss # ここでlossを消した方がGPUのメモリの無駄な部分を消せるらしい。https://tma15.github.io/blog/2020/08/22/pytorch%E4%B8%8D%E8%A6%81%E3%81%AB%E3%81%AA%E3%81%A3%E3%81%9F%E8%A8%88%E7%AE%97%E3%82%B0%E3%83%A9%E3%83%95%E3%82%92%E5%89%8A%E9%99%A4%E3%81%97%E3%81%A6%E3%83%A1%E3%83%A2%E3%83%AA%E3%82%92%E7%AF%80%E7%B4%84/
        
#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()
#         if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
#             print('Epoch: [{0}][{1}/{2}] '
#                   'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
#                   'Elapsed {remain:s} '
#                   'Loss: {loss.val:.4f}({loss.avg:.4f}) '
#                   'Grad: {grad_norm:.4f}  '
#                   #'LR: {lr:.6f}  '
#                   .format(
#                    epoch+1, step, len(train_loader), batch_time=batch_time,
#                    data_time=data_time, loss=losses,
#                    remain=timeSince(start, float(step+1)/len(train_loader)),
#                    grad_norm=grad_norm,
#                    #lr=scheduler.get_lr()[0],
#                    ))

#     # データローダーの合併
#     allpreds = np.concatenate(allpreds)
#     alltargets = np.concatenate(alltargets)


#     # Score with rmse
#     train_rmse = np.sqrt(mean_squared_error(alltargets,allpreds))
    
#     return losses.avg, train_rmse

In [23]:
def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    
    model.eval()
    start = end = time.time()

    allpreds = []
    alltargets = []

    for step, a in enumerate(valid_loader):

        with torch.no_grad():

            ids = a["ids"].to(device)
            mask = a["mask"].to(device)
            batch_size = ids.size(0)

            output = model(ids,mask)
            output = output["logits"].squeeze(-1)

            target = a["targets"].to(device)

            loss = criterion(output,target)


            # 採点用に
            losses.update(loss.item(), batch_size)
            allpreds.append(output.detach().cpu().numpy())
            alltargets.append(target.detach().squeeze(-1).cpu().numpy())
            
            del loss
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))



    # dataloader分を結合

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)


    # rmseでスコア出し
    valid_rme_loss = np.sqrt(mean_squared_error(alltargets,allpreds))

    return allpreds,losses.avg,valid_rme_loss

In [24]:
# def valid_fn(valid_loader, model, criterion, device):
#     batch_time = AverageMeter()
#     data_time = AverageMeter()
#     losses = AverageMeter()
#     scores = AverageMeter()
    
#     # switch to evaluation mode
#     model.eval()
#     allpreds = []
#     alltargets = []
#     start = end = time.time()

#     for step, a in enumerate(valid_loader):
#         # measure data loading time
#         data_time.update(time.time() - end)
#         with torch.no_grad():
#             ids = a["ids"].to(device)
#             mask = a["mask"].to(device)

#             batch_size = ids.size(0)

#             y_preds = model(ids, mask)["logits"]
        
#             target = a["targets"].to(device)
#             loss = criterion(y_preds.squeeze(-1), target)
#             losses.update(loss.item(), batch_size)
                    
#             # record accuracy
#             allpreds.append(y_preds.detach().cpu().numpy())
#             alltargets.append(target.detach().squeeze(-1).cpu().numpy())        
#             if CFG.gradient_accumulation_steps > 1:
#                 loss = loss / CFG.gradient_accumulation_steps
#             del loss

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()
#         if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
#             print('EVAL: [{0}/{1}] '
#                   'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
#                   'Elapsed {remain:s} '
#                   'Loss: {loss.val:.4f}({loss.avg:.4f}) '
#                   .format(
#                    step, len(valid_loader), batch_time=batch_time,
#                    data_time=data_time, loss=losses,
#                    remain=timeSince(start, float(step+1)/len(valid_loader)),
#                    ))
#     # dataloader分を結合

#     allpreds = np.concatenate(allpreds)
#     alltargets = np.concatenate(alltargets)

#     # rmseでスコア出し
#     valid_rmse = np.sqrt(mean_squared_error(alltargets,allpreds))

#     return allpreds, losses.avg, valid_rmse

# Train loop

In [25]:
# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_col].values

    train_dataset = RobertaDataSet(train_folds["excerpt"],train_folds[CFG.target_col])
    valid_dataset = RobertaDataSet(valid_folds["excerpt"],valid_folds[CFG.target_col])

    train_loader = DataLoader(train_dataset,
                                  batch_size = CFG.batch_size,
                                  shuffle = True,
                                  num_workers=CFG.num_workers,
                                  pin_memory=True)
    valid_loader = DataLoader(valid_dataset,
                                  batch_size=CFG.batch_size * 2,
                                  shuffle = False,
                                  num_workers=CFG.num_workers,
                                  pin_memory=True)
        
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='LinearSchedule':
            train_steps = int(len(train_folds)/CFG.batch_size*CFG.epochs)
            num_steps = int(train_steps*0.1)
            scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)
            
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
    model = transformers.RobertaForSequenceClassification.from_pretrained("/content/roberta-base",num_labels=1)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=CFG.lr, betas=(0.9, 0.999), weight_decay=CFG.weight_decay) # AdamW optimizer
    scheduler = get_scheduler(optimizer)
    
    # ====================================================
    # loop
    # ====================================================
    criterion = RMSELoss()

    best_score = np.inf
    best_loss = np.inf

    
# ここから修正頑張る
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss, train_rmse = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        preds, avg_val_loss, valid_rmse = valid_fn(valid_loader, model, criterion, device)
        
        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {valid_rmse:.4f}')

        if valid_rmse < best_score:
            best_score = valid_rmse
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        MODEL_DIR+f'{CFG.model_name}_fold{fold}_best_score.pth')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        MODEL_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth')
    
    valid_folds['preds'] = torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth', 
                                      map_location=torch.device('cpu'))['preds']

    return valid_folds

In [26]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
        return score
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        bestscores = []
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                score = get_result(_oof_df)
                bestscores.append(score)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)
        print(bestscores)

In [27]:
if __name__ == '__main__':
    main()

========== fold: 0 training ==========
Some weights of the model checkpoint at /content/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/roberta-base and are 

Epoch: [1][0/142] Data 0.000 (0.000) Elapsed 0m 0s (remain 2m 10s) Loss: 1.4137(1.4137) 
Epoch: [1][50/142] Data 0.000 (0.000) Elapsed 0m 8s (remain 0m 14s) Loss: 1.3274(1.4843) 
Epoch: [1][100/142] Data 0.000 (0.000) Elapsed 0m 15s (remain 0m 6s) Loss: 0.7630(1.2392) 
Epoch: [1][141/142] Data 0.000 (0.000) Elapsed 0m 21s (remain 0m 0s) Loss: 0.7850(1.0827) 
EVAL: [0/18] Data 0.000 (0.000) Elapsed 0m 0s (remain 0m 10s) Loss: 0.9726(0.9726) 


Epoch 1 - avg_train_loss: 1.0827  avg_val_loss: 0.5834  time: 25s
Epoch 1 - Score: 0.6178
Epoch 1 - Save Best Score: 0.6178 Model


EVAL: [17/18] Data 0.000 (0.000) Elapsed 0m 3s (remain 0m 0s) Loss: 1.0648(0.5834) 


Epoch 1 - Save Best Loss: 0.5834 Model


...

EVAL: [17/18] Data 0.000 (0.000) Elapsed 0m 3s (remain 0m 0s) Loss: 0.6170(0.5505) 
Epoch: [20][0/142] Data 0.000 (0.000) Elapsed 0m 0s (remain 1m 11s) Loss: 0.0596(0.0596) 
Epoch: [20][50/142] Data 0.000 (0.000) Elapsed 0m 7s (remain 0m 13s) Loss: 0.0798(0.1138) 
Epoch: [20][100/142] Data 0.000 (0.000) Elapsed 0m 15s (remain 0m 6s) Loss: 0.0997(0.1155) 
Epoch: [20][141/142] Data 0.000 (0.000) Elapsed 0m 20s (remain 0m 0s) Loss: 0.1073(0.1150) 
EVAL: [0/18] Data 0.000 (0.000) Elapsed 0m 0s (remain 0m 11s) Loss: 0.6511(0.6511) 


Epoch 20 - avg_train_loss: 0.1150  avg_val_loss: 0.5487  time: 25s
Epoch 20 - Score: 0.5572


EVAL: [17/18] Data 0.000 (0.000) Elapsed 0m 3s (remain 0m 0s) Loss: 0.6192(0.5487) 


========== fold: 4 result ==========
Score: 0.4968
========== CV ==========
Score: 0.5301


[0.5354156179873284, 0.5417753295203239, 0.5483728280856506, 0.5266642623808407, 0.4968084838712649]


# 